# Data Structuring

In [2]:
import pandas as pd
import numpy as np

data = pd.read_csv('data/sleep_wake.csv')
data.set_index('Date', inplace=True, drop=True)
data.index=data.index.to_datetime(dayfirst=False)
data = data.ix[:, 0:2]

data['Sleep_time'] = [entry[0] + pd.DateOffset(minutes=entry[1], hours=22) for entry in zip(data.index, data['Sleep'])]
data['Wake_time'] = [entry[0] + pd.DateOffset(minutes=entry[1], hours=6) for entry in zip(data.index, data['Wake'])]
data.head()

,Sleep,Wake,Sleep_time,Wake_time
2017-12-01,16,1,2017-12-01 22:16:00,2017-12-01 06:01:00
2017-12-02,19,-10,2017-12-02 22:19:00,2017-12-02 05:50:00
2017-12-03,-45,0,2017-12-03 21:15:00,2017-12-03 06:00:00
2017-12-04,8,1,2017-12-04 22:08:00,2017-12-04 06:01:00
2017-12-05,-23,-16,2017-12-05 21:37:00,2017-12-05 05:44:00


In [3]:
# Create a separate sleep dataframe
sleep_index = pd.date_range(start=min(data.index), end=max(data.index), freq='60S')
sleep_index = [time for time in sleep_index if (time.hour >= 21) & (time.hour <= 23)]
sleep_data = pd.DataFrame(0, columns=['indicator'], index=sleep_index)

# Set an index with an indicator for times when I am asleep
# Indicator = 1 when I am asleep, else 0
for time in sleep_data.index:
    for sleep_time in data['Sleep_time']:
        if time == sleep_time:
            selected_date = time.date()
            date_subset = [date for date in sleep_data.index if date.date() == selected_date]
            asleep_times = [time for time in date_subset if time.time() >= sleep_time.time()]
            sleep_data.ix[asleep_times, 'indicator'] = 1
            
# Create a separate wake dataframe
wake_index=pd.date_range(start=min(data.index), end=max(data.index), freq='60S')
wake_index = [time for time in wake_index if (time.hour <= 7) & (time.hour >= 5)]
wake_data = pd.DataFrame(1, columns=['indicator'], index=wake_index)

# Set an index with indicator for times when I am asleep
# Indicator = 1 when I am asleep, else 0
for time in wake_data.index:
    for wake_time in data['Wake_time']:
        if time == wake_time:
            selected_date = time.date()
            date_subset = [date for date in wake_data.index if date.date() == selected_date]
            wake_times = [time for time in date_subset if time.time() >= wake_time.time()]
            wake_data.ix[wake_times, 'indicator'] = 0
    
# Time offset column 
# For sleep, offset from 10:00 PM
sleep_data['time_offset'] = [int((time.hour * 60 + time.minute) - (22 * 60)) for time in sleep_data.index]
# For wake, offset from 6:00 AM
wake_data['time_offset'] = [int((time.hour * 60 + time.minute) - (6 * 60)) for time in wake_data.index]

## Write Data to a csv 

In [5]:
sleep_data.to_csv('data/sleep_data.csv')
wake_data.to_csv('data/wake_data.csv')